In [2]:
# List all NVIDIA GPUs as avaialble in this computer (or Colab's session)
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-08be3f7b-db7d-7532-32a7-f6624aa03ab0)


In [3]:
import sys
print( f"Python {sys.version}\n" )

import numpy as np
print( f"NumPy {np.__version__}\n" )

import matplotlib.pyplot as plt
%matplotlib inline

import tensorflow as tf
print( f"TensorFlow {tf.__version__}" )
print( f"tf.keras.backend.image_data_format() = {tf.keras.backend.image_data_format()}" )

# Count the number of GPUs as detected by tensorflow
gpus = tf.config.list_physical_devices('GPU')
print( f"TensorFlow detected { len(gpus) } GPU(s):" )
for i, gpu in enumerate(gpus):
  print( f".... GPU No. {i}: Name = {gpu.name} , Type = {gpu.device_type}" )

Python 3.8.16 (default, Dec  7 2022, 01:12:13) 
[GCC 7.5.0]

NumPy 1.21.6

TensorFlow 2.9.2
tf.keras.backend.image_data_format() = channels_last
TensorFlow detected 1 GPU(s):
.... GPU No. 0: Name = /physical_device:GPU:0 , Type = GPU


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
import re
import csv
import os
import pyarrow as pa
import torch
import pyarrow as pa
from datasets import DatasetDict
from datasets.arrow_dataset import Dataset

In [7]:
List_key = []
# Open the CSV file for reading
for i in range(1000): 
  with open('/content/drive/MyDrive/DADS7202_final/Chart2text/data/'+str(i)+'.csv', 'r') as d:
 # Read the contents of the file into a list of rows
    rows = list(csv.reader(d))
# Concatenate the rows into a single string
    csv_string = '\n'.join(['\t'.join(row) for row in rows])
    csv_string = re.sub('\s+',' ',csv_string)
    List_key.append(csv_string)
len(List_key)

1000

In [8]:
txt_key = []
for i in range(1000): 
 f = open('/content/drive/MyDrive/DADS7202_final/Chart2text/captions/'+str(i)+'.txt', 'r')
 content = f.read()
 content = re.sub('\s+',' ',content)
 txt_key.append(content)
len(txt_key)

1000

In [9]:
print(List_key[0])

Quarter Number of users in millions Q4 '19 2498 Q3 '19 2449 Q2 '19 2414 Q1 '19 2375 Q4 '18 2320 Q3 '18 2271 Q2 '18 2234 Q1 '18 2196 Q4 '17 2129 Q3 '17 2072 Q2 '17 2006 Q1 '17 1936 Q4 '16 1860 Q3 '16 1788 Q2 '16 1712 Q1 '16 1654 Q4 '15 1591 Q3 '15 1545 Q2 '15 1490 Q1 '15 1441 Q4 '14 1393 Q3 '14 1350 Q2 '14 1317 Q1 '14 1276 Q4 '13 1228 Q3 '13 1189 Q2 '13 1155 Q1 '13 1110 Q4 '12 1056 Q3 '12 1007 Q2 '12 955 Q1 '12 901 Q4 '11 845 Q3 '11 800 Q2 '11 739 Q1 '11 680 Q4 '10 608 Q3 '10 550 Q2 '10 482 Q1 '10 431 Q4 '09 360 Q3 '09 305 Q2 '09 242 Q1 '09 197 Q3 '08 100


In [10]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [30]:
# Create a list of lists that represents the data
train_data = [List_key[:600] ,txt_key[:600]]

# Create a list of pyarrow.Array objects from the data
arrays_train = [pa.array(col) for col in train_data]

# Create a pyarrow Table object from the arrays
table1 = pa.Table.from_arrays(arrays_train, names=['key', 'text'])
#print(type(table1))

# Create a Dataset object from the table
dataset1 = Dataset(table1)
#print(type(dataset1))

# Create a list of lists that represents the data
val_data = [List_key[600:800] ,txt_key[600:800]]

# Create a list of pyarrow.Array objects from the data
arrays_val = [pa.array(col) for col in val_data]

# Create a pyarrow Table object from the arrays
table2 = pa.Table.from_arrays(arrays_val, names=['key', 'text'])
#print(type(table1))

# Create a Dataset object from the table
dataset2 = Dataset(table2)
print(type(dataset2))

# Create a list of lists that represents the data
test_data = [List_key[800:] ,txt_key[800:]]

# Create a list of pyarrow.Array objects from the data
arrays_test = [pa.array(col) for col in test_data]

# Create a pyarrow Table object from the arrays
table3 = pa.Table.from_arrays(arrays_test, names=['key', 'text'])
#print(type(table1))

# Create a Dataset object from the table
dataset3 = Dataset(table3)
print(type(dataset3))

# Create a dictionary that maps keys to the Dataset objects
dataset = {
    'train': dataset1,
    'validation': dataset2,
    'test': dataset3
}

# Access a Dataset object using its key
print(dataset)  # Output: Dataset object with train_data
print(type(dataset))
#print(dataset['validation'])  # Output: Dataset object with validation_data
#print(dataset['test'])  # Output: Dataset object with test_data

<class 'datasets.arrow_dataset.Dataset'>
<class 'datasets.arrow_dataset.Dataset'>
{'train': Dataset({
    features: ['key', 'text'],
    num_rows: 600
}), 'validation': Dataset({
    features: ['key', 'text'],
    num_rows: 200
}), 'test': Dataset({
    features: ['key', 'text'],
    num_rows: 200
})}
<class 'dict'>


In [31]:
from datasets import DatasetDict

dataset_dict = DatasetDict(dataset)
print(type(dataset_dict))
print(dataset_dict)

<class 'datasets.dataset_dict.DatasetDict'>
DatasetDict({
    train: Dataset({
        features: ['key', 'text'],
        num_rows: 600
    })
    validation: Dataset({
        features: ['key', 'text'],
        num_rows: 200
    })
    test: Dataset({
        features: ['key', 'text'],
        num_rows: 200
    })
})


In [32]:
print(dataset['train'][1])

{'key': "NBA player Number of points scored Kareem Abdul-Jabbar 38387 Karl Malone 36928 LeBron James* 33655 Kobe Bryant 33643 Michael Jordan 32292 Dirk Nowitzki 31560 Wilt Chamberlain 31419 Shaquille O'Neal 28596 Moses Malone 27409 Elvin Hayes 27313 Hakeem Olajuwon 26946 Oscar Robertson 26710 Dominique Wilkins 26668 Tim Duncan 26496 Paul Pierce 26397 John Havlicek 26395 Kevin Garnett 26071 Carmelo Anthony* 26067 Vince Carter* 25623 Alex English 25613 Reggie Miller 25279 Jerry West 25192 Patrick Ewing 24815 Ray Allen 24505 Allen Iverson 24368 Charles Barkley 23757 Robert Parish 23334 Adrian Dantley 23177 Dwyane Wade 23165 Elgin Baylor 23149 Kevin Durant* 22940", 'text': "How many points does LeBron James have ? In January 2020 , LeBron James passed Kobe Bryant on the NBA all-time scoring list , scoring 29 points to in a regular season game against the Philadelphia 76ers . If James continues to produce at about his career average , he has a shot to move up even further and pass Karl Malo

In [33]:
example = dataset_dict['train'][1]

print("Key:", example["key"])
print("Text:", example["text"])

Key: NBA player Number of points scored Kareem Abdul-Jabbar 38387 Karl Malone 36928 LeBron James* 33655 Kobe Bryant 33643 Michael Jordan 32292 Dirk Nowitzki 31560 Wilt Chamberlain 31419 Shaquille O'Neal 28596 Moses Malone 27409 Elvin Hayes 27313 Hakeem Olajuwon 26946 Oscar Robertson 26710 Dominique Wilkins 26668 Tim Duncan 26496 Paul Pierce 26397 John Havlicek 26395 Kevin Garnett 26071 Carmelo Anthony* 26067 Vince Carter* 25623 Alex English 25613 Reggie Miller 25279 Jerry West 25192 Patrick Ewing 24815 Ray Allen 24505 Allen Iverson 24368 Charles Barkley 23757 Robert Parish 23334 Adrian Dantley 23177 Dwyane Wade 23165 Elgin Baylor 23149 Kevin Durant* 22940
Text: How many points does LeBron James have ? In January 2020 , LeBron James passed Kobe Bryant on the NBA all-time scoring list , scoring 29 points to in a regular season game against the Philadelphia 76ers . If James continues to produce at about his career average , he has a shot to move up even further and pass Karl Malone with a

In [15]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [34]:
from transformers import AutoTokenizer, AutoModelWithLMHead, AutoModelForSeq2SeqLM

In [35]:
tokenizer = AutoTokenizer.from_pretrained("t5-small")

prefix = "Text generation from keyword: "
max_input_length = 256
max_target_length = 128

def preprocess_examples(examples):
  # encode the code-docstring pairs
  codes = examples['key']
  docstrings = examples['text']
  
  inputs = [prefix + code for code in codes]
  model_inputs = tokenizer(inputs, max_length=max_input_length, padding="max_length", truncation=True)

  # encode the summaries
  labels = tokenizer(docstrings, max_length=max_target_length, padding="max_length", truncation=True).input_ids

  # important: we need to replace the index of the padding tokens by -100
  # such that they are not taken into account by the CrossEntropyLoss
  labels_with_ignore_index = []
  for labels_example in labels:
    labels_example = [label if label != 0 else -100 for label in labels_example]
    labels_with_ignore_index.append(labels_example)
  
  model_inputs["labels"] = labels_with_ignore_index

  return model_inputs

/usr/local/lib/python3.8/dist-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-small automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [36]:
dataset = dataset_dict.map(preprocess_examples, batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [37]:
from torch.utils.data import DataLoader

dataset.set_format(type="torch", columns=['input_ids', 'attention_mask', 'labels'])
train_dataloader = DataLoader(dataset['train'], shuffle=True, batch_size=8)
valid_dataloader = DataLoader(dataset['validation'], batch_size=4)
test_dataloader = DataLoader(dataset['test'], batch_size=4)

In [38]:
batch = next(iter(train_dataloader))
print(batch.keys())

dict_keys(['input_ids', 'attention_mask', 'labels'])


In [20]:
!pip install -q pytorch-lightning wandb

     |████████████████████████████████| 800 kB 30.6 MB/s 
     |████████████████████████████████| 1.9 MB 60.8 MB/s 
     |████████████████████████████████| 512 kB 83.9 MB/s 
     |████████████████████████████████| 125 kB 77.2 MB/s 
     |████████████████████████████████| 182 kB 86.0 MB/s 
     |████████████████████████████████| 174 kB 82.5 MB/s 
     |████████████████████████████████| 62 kB 1.5 MB/s 
     |████████████████████████████████| 173 kB 83.7 MB/s 
     |████████████████████████████████| 168 kB 93.8 MB/s 
     |████████████████████████████████| 168 kB 83.4 MB/s 
     |████████████████████████████████| 166 kB 87.6 MB/s 
     |████████████████████████████████| 166 kB 95.4 MB/s 
     |████████████████████████████████| 162 kB 92.9 MB/s 
     |████████████████████████████████| 162 kB 86.1 MB/s 
     |████████████████████████████████| 158 kB 89.7 MB/s 
     |████████████████████████████████| 157 kB 94.6 MB/s 
     |████████████████████████████████| 157 kB 88.5 MB/s 
     |██████████

In [39]:
from transformers import T5ForConditionalGeneration, AdamW, get_linear_schedule_with_warmup
import pytorch_lightning as pl

class CodeT5(pl.LightningModule):
    def __init__(self, lr=5e-5, num_train_epochs=100, warmup_steps=100):
        super().__init__()
        self.model = AutoModelForSeq2SeqLM.from_pretrained("t5-small")
        self.save_hyperparameters()

    def forward(self, input_ids, attention_mask, labels=None):     
        outputs = self.model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        return outputs
    
    def common_step(self, batch, batch_idx):
        outputs = self(**batch)
        loss = outputs.loss

        return loss
      
    def training_step(self, batch, batch_idx):
        loss = self.common_step(batch, batch_idx)     
        # logs metrics for each training_step,
        # and the average across the epoch
        self.log("training_loss", loss)

        return loss

    def validation_step(self, batch, batch_idx):
        loss = self.common_step(batch, batch_idx)     
        self.log("validation_loss", loss, on_epoch=True)

        return loss

    def test_step(self, batch, batch_idx):
        loss = self.common_step(batch, batch_idx)     

        return loss

    def configure_optimizers(self):
        # create optimizer
        optimizer = AdamW(self.parameters(), lr=self.hparams.lr)
        # create learning rate scheduler
        num_train_optimization_steps = self.hparams.num_train_epochs * len(train_dataloader)
        lr_scheduler = {'scheduler': get_linear_schedule_with_warmup(optimizer,
                                                    num_warmup_steps=self.hparams.warmup_steps,
                                                    num_training_steps=num_train_optimization_steps),
                        'name': 'learning_rate',
                        'interval':'step',
                        'frequency': 1}
        
        return {"optimizer": optimizer, "lr_scheduler": lr_scheduler}

    def train_dataloader(self):
        return train_dataloader

    def val_dataloader(self):
        return valid_dataloader

    def test_dataloader(self):
        return test_dataloader

In [40]:
import wandb

wandb.login()

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

In [41]:
tokenizer.decode(batch['input_ids'][0])

'Text generation from keyword: Year Number of households in millions 2019 128.58 2018 127.59 2017 126.22 2016 125.82 2015 124.59 2014 123.23 2013 122.46 2012 121.08 2011 118.68 2010 117.54 2009 117.18 2008 116.78 2007 116.01 2006 114.38 2005 113.34 2004 112.0 2003 111.28 2002 109.3 2001 108.21 2000 104.71 1999 103.87 1998 102.53 1997 101.02 1996 99.63 1995 98.99 1994 97.11 1993 96.39 1992 95.67 1991 94.31 1990 93.35 1989 92.83 1988 91.07 1987 89.48 1986 88.46 1985 86.79 1984 85.41 1983 83.92 1982 83.53 1981 82.37 1980 80.78 1979 77.33 1978 76.03 1977 74.14 1976 72.87 1975 71.12 1974 69.86 1973 68.25 1972 66.68 1971 64.78 1970 63.4 1960 52.8</s><pad><pad><pad><pad>'

In [42]:
labels = batch['labels'][0]
tokenizer.decode([label for label in labels if label != -100])

'How many households are in the U.S.? In 2019, there were 128.58 million households in the United States. This is a significant increase from 1960, when there were 52.8 million households in the U.S. What counts as a household? According to the U.S. Census Bureau, a household is considered to be all persons living within one housing unit. This includes apartments, houses, or single rooms, and consists of both related and unrelated people living together. For example, two roommates who share a living space</s>'

In [43]:
model = CodeT5()

In [44]:
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor

wandb_logger = WandbLogger(name='codet5-finetune-code-summarization-ruby-shuffle', project='CodeT5')
# for early stopping, see https://pytorch-lightning.readthedocs.io/en/1.0.0/early_stopping.html?highlight=early%20stopping
early_stop_callback = EarlyStopping(
    monitor='validation_loss',
    patience=3,
    strict=False,
    verbose=False,
    mode='min'
)
lr_monitor = LearningRateMonitor(logging_interval='step')

trainer = Trainer(gpus=1, 
                  default_root_dir="/content/drive/MyDrive/CodeT5/Notebooks/Checkpoints", 
                  logger=wandb_logger, 
                  callbacks=[early_stop_callback, lr_monitor])
trainer.fit(model)

/usr/local/lib/python3.8/dist-packages/pytorch_lightning/loggers/wandb.py:396: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
/usr/local/lib/python3.8/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:604: UserWarning: Checkpoint directory ./CodeT5/30j54ybj/checkpoints exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.mod

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

In [45]:
save_directory = "/content/drive/MyDrive/DADS7202_final/model_t5-small_1000_100e" # save in the current working directory, you can change this of course
model.model.save_pretrained(save_directory)

In [46]:
dataset2 = dataset_dict
print(dataset2['test'])

Dataset({
    features: ['key', 'text'],
    num_rows: 200
})


In [47]:
test_example = dataset2['test'][0]
print("Key:", test_example['key'])

Key: Year Share of respondents 2019 73% 2018 69% 2017 63% 2016 60% 2015 55% 2014 53% 2013 50% 2012 47% 2011 44% 2010 42% 2009 41% 2008 35% 2007 30%


In [ ]:
#from transformers import T5ForConditionalGeneration
#model = T5ForConditionalGeneration.from_pretrained(save_directory)

In [48]:
model = AutoModelForSeq2SeqLM.from_pretrained(save_directory)

In [49]:
# prepare for the model
input_ids = tokenizer(test_example['key'], return_tensors='pt').input_ids
# generate
outputs = model.generate(input_ids,min_length=120, max_length=150)
print("Generated docstring:", tokenizer.decode(outputs[0], skip_special_tokens=True))

Generated docstring: The statistic shows the share of respondents from 2007 to 2019. In 2019, the share of respondents was 73% of respondents. The survey shows that the majority of respondents were in the United States. The majority of respondents were in the United States, with a share of 73 percent in 2019. The majority of respondents were in the United States, with a share of 73 percent in 2019. The majority of respondents were in the United States, with a share of 63 percent in 2019. The majority of respondents were born in the United States. The


In [53]:
'''#prepare for the model
input_ids = tokenizer('Foreign students in English-speaking countries Number of students in 1000s 2004 2012 USA 175000 245000 UK 120000 200000 Australia 120000 200000', return_tensors='pt').input_ids
# generate
outputs = model.generate(input_ids,min_length=120, max_length=150)
print("Generated docstring:", tokenizer.decode(outputs[0], skip_special_tokens=True))'''

'#prepare for the model\ninput_ids = tokenizer(\'Foreign students in English-speaking countries Number of students in 1000s 2004 2012 USA 175000 245000 UK 120000 200000 Australia 120000 200000\', return_tensors=\'pt\').input_ids\n# generate\noutputs = model.generate(input_ids,min_length=120, max_length=150)\nprint("Generated docstring:", tokenizer.decode(outputs[0], skip_special_tokens=True))'

In [51]:
test_example = dataset2['test'][0]
print("Text:", test_example['text'])

Text: This statistic illustrates the share of individuals using online banking regularly in Great Britain from 2007 to 2018 . Banking and financial services are typically among the first industries to embrace technological advances ; starting as far back in time as the appearance of automated teller machines ( ATMs ) and credit cards . Ever since the first secure Internet systems for financial operations became available on a large scale , accessing of one 's bank account online to carry out transactions , pay off credit cards , establish credit lines or invest savings became one of the fastest growing areas of Internet activities . Mobile banking has become the new norm In the UK , online banking use increased during the period under observation : already 30 percent of respondents reported accessing their bank account online within three months prior to the survey in 2007 . The growth in this activity area was steady over time , and the share rose to 69 percent of individuals who used

In [54]:
test_example = dataset2['test'][0]
print("Key:", test_example['key'])

Key: Year Share of respondents 2019 73% 2018 69% 2017 63% 2016 60% 2015 55% 2014 53% 2013 50% 2012 47% 2011 44% 2010 42% 2009 41% 2008 35% 2007 30%
